In [2]:
!pip install torch torchvision torchaudio

Defaulting to user installation because normal site-packages is not writeable


In [3]:
%pip3 install transformers requests beautifulsoup4 pandas numpy

UsageError: Line magic function `%pip3` not found.


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

/Users/pansykuang/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/pansykuang/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
#instantiate model
tokenizer=AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [ ]:
#collect reviews from yelp

#make request to site
r= requests.get('https://www.yelp.com/biz/happy-feet-massage-belmont')
#extract comments from yelp site
soup = BeautifulSoup(r.text,'html.parser')
regex= re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]


for i in range(10,110,10):
    r= requests.get('https://www.yelp.com/biz/happy-feet-massage-belmont?osq=happy+feet+massage&start='+str(i))
    soup = BeautifulSoup(r.text,'html.parser')
    regex= re.compile('.*comment.*')
    results = soup.find_all('p', {'class':regex})
    reviews +=[result.text for result in results]

In [28]:
len(reviews)

103

In [29]:
#load reviews into dataframe and score
import pandas as pd
import numpy as np

df = pd.DataFrame(np.array(reviews), columns=['review'])
df

,review
0,"Happy Feet Massage240 El Camino RealBelmont, C..."
1,I have been coming here since before the pande...
2,Foot reflexology places are similar to the Thr...
3,"Though I'm far from a regular, I've had probab..."
4,I had an amazing experience at this massage pl...
...,...
98,Happy feet massage is the best one ! People ar...
99,I thoroughly enjoyed my massage's that I get h...
100,Quite possible the worst massage I have ever h...
101,Consistently good massage at a reasonable pric...


In [30]:
#gets sentiment from string using encoding logic
def get_sentiment(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

#put sentiment score column on review dataframe
df['sentiment'] = df['review'].apply(lambda x: get_sentiment(x[:512]))
df


,review,sentiment
0,"Happy Feet Massage240 El Camino RealBelmont, C...",5
1,I have been coming here since before the pande...,5
2,Foot reflexology places are similar to the Thr...,3
3,"Though I'm far from a regular, I've had probab...",4
4,I had an amazing experience at this massage pl...,5
...,...,...
98,Happy feet massage is the best one ! People ar...,5
99,I thoroughly enjoyed my massage's that I get h...,5
100,Quite possible the worst massage I have ever h...,1
101,Consistently good massage at a reasonable pric...,4
